# Data Preprocessing

In [ ]:
import os

import numpy as np
import pandas as pd

In [2]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
data_dir = "data"
file_path = os.path.join(data_dir, "merged_data.csv")
stock_data = pd.read_csv(file_path)

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29661 entries, 0 to 29660
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   symbol      29661 non-null  object
 1   market_cap  29661 non-null  object
 2   date        29661 non-null  object
 3   open        29661 non-null  object
 4   high        29661 non-null  object
 5   low         29661 non-null  object
 6   close       29661 non-null  object
 7   adj_close   29661 non-null  object
 8   change      29661 non-null  object
 9   volume      29661 non-null  object
dtypes: object(10)
memory usage: 2.3+ MB


In [ ]:
stock_data.sample(10)

,symbol,market_cap,date,open,high,low,close,adj_close,change,volume
15556,ATR,Mid-Cap,"Dec 18, 2024",163.69,164.42,157.91,158.21,157.76,-2.99%,"468,597"
11398,FSTR,Micro-Cap,"Oct 22, 2024",19.73,19.79,19.51,19.63,19.63,-0.46%,"24,175"
5866,AZN,Mega-Cap,"Nov 7, 2024",63.51,64.74,63.37,64.69,64.69,1.32%,"14,404,312"
17081,G,Mid-Cap,"Nov 12, 2024",46.47,46.91,46.06,46.67,46.51,0.02%,"1,998,849"
21778,JRSH,Nano-Cap,"Nov 15, 2024",3.27,3.35,3.15,3.3,3.25,-,"8,266"
17671,KNTK,Mid-Cap,"Nov 26, 2024",59.21,60.0,58.37,59.16,58.45,-0.03%,"642,022"
26985,LEU,Small-Cap,"Jan 10, 2025",72.42,74.66,69.5,70.83,70.83,-2.20%,"379,005"
14014,SFST,Micro-Cap,"Sep 26, 2024",33.31,33.34,32.98,32.99,32.99,0.15%,"9,953"
12895,MVIS,Micro-Cap,"Oct 23, 2024",1.11,1.12,1.07,1.1,1.1,-0.90%,"1,552,104"
7610,LLY,Mega-Cap,"Jan 31, 2025",823.0,827.35,811.03,811.08,809.64,-1.48%,"2,562,617"


## Data Cleaning

In [ ]:
stock_data["date"] = pd.to_datetime(stock_data["date"], format="%b %d, %Y")

In [ ]:
for column in stock_data.columns:
    if stock_data[column].dtype == object:
        stock_data[column] = stock_data[column].str.replace(",", "", regex=True)
        try:
            stock_data[column] = stock_data[column].astype(float)
        except ValueError:
            pass

In [ ]:
stock_data["change"] = stock_data["change"].replace("-", np.nan)
stock_data["change"] = stock_data["change"].str.replace("%", "").astype(float)
stock_data["change"] = stock_data["change"] / 100

In [ ]:
stock_data["volume"] = stock_data["volume"].str.replace(",", "")
stock_data["volume"] = stock_data["volume"].replace("-", np.nan)
stock_data["volume"] = stock_data["volume"].astype(float)

In [ ]:
stock_data.sort_values(by=["symbol", "date"], inplace=True)
stock_data["calculated_change"] = stock_data.groupby("symbol")["close"].pct_change()
stock_data["change"].fillna(stock_data["calculated_change"], inplace=True)
stock_data.drop(columns=["calculated_change"], inplace=True)
stock_data["change"].fillna(method="ffill", inplace=True)
stock_data["volume"].fillna(method="ffill", inplace=True)

In [ ]:
stock_data["symbol"] = stock_data["symbol"].str.upper()

In [ ]:
stock_data["market_cap"] = stock_data["market_cap"].str.strip()

In [ ]:
stock_data.duplicated().sum()

np.int64(0)

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29661 entries, 14817 to 24565
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   symbol      29661 non-null  object        
 1   market_cap  29661 non-null  object        
 2   date        29661 non-null  datetime64[ns]
 3   open        29661 non-null  float64       
 4   high        29661 non-null  float64       
 5   low         29661 non-null  float64       
 6   close       29661 non-null  float64       
 7   adj_close   29661 non-null  float64       
 8   change      29661 non-null  float64       
 9   volume      29661 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 2.5+ MB


In [ ]:
stock_data.sample(10)

,symbol,market_cap,date,open,high,low,close,adj_close,change,volume
28487,SEZL,Small-Cap,2025-01-07,281.50,281.50,250.11,254.19,254.19,-0.0792,148227.0
14894,AGI,Mid-Cap,2024-10-24,21.20,21.27,20.62,21.09,21.06,-0.0005,2445209.0
21381,GWH,Nano-Cap,2024-11-12,9.85,9.85,8.30,9.10,9.10,-0.0761,184650.0
13257,OSUR,Micro-Cap,2024-12-17,3.97,4.00,3.83,3.87,3.87,-0.0252,898572.0
15573,ATR,Mid-Cap,2024-11-22,170.18,172.07,170.18,171.51,171.02,0.0096,241648.0
21408,GWH,Nano-Cap,2024-10-04,6.36,6.53,6.14,6.52,6.52,0.0399,48912.0
29298,UFPT,Small-Cap,2024-12-19,243.19,248.32,238.36,246.04,246.04,0.0172,102122.0
1514,DHR,Large-Cap,2025-01-27,247.52,250.76,246.46,250.59,250.59,0.0195,2748518.0
18302,MUSA,Mid-Cap,2024-10-14,490.69,494.56,487.49,489.43,488.94,-0.0012,120440.0
2951,PFE,Large-Cap,2024-11-29,25.83,26.23,25.77,26.21,25.79,0.0147,22023022.0


In [ ]:
save_to_path = os.path.join(data_dir, "cleaned_merged_data.csv")
stock_data.to_csv(save_to_path, index=False)

---